In [14]:
import pandas as pd
import numpy as np
import networkx as nx
import statsmodels.formula.api as smf

In [2]:
# Load datasets
africa_visa_openness_2019 = pd.read_csv("africa_visa_openness_2019.csv")
africa_visa_openness_2023 = pd.read_csv("africa_visa_openness_2023.csv")
passport_index_2019 = pd.read_csv("passport-index-2019.csv")
passport_index_2023 = pd.read_csv("passport-index-2023.csv")
henley_passport_index_2025 = pd.read_csv("henley_passport_index_2025.csv")
gdp_per_capita_2019_2023 = pd.read_csv("gdp_per_capita_2019_2023.csv")
country_region_income_group = pd.read_csv("country_region_income_group.csv")

In [3]:
# Rename columns
africa_visa_openness_2019 = africa_visa_openness_2019.rename(columns={
    "Country": "country",
    "ISO3": "iso3",
    "No visa required": "avoi_no_visa_2019",
    "Visa on arrival": "avoi_visa_on_arrival_2019",
    "Visa required": "avoi_visa_required_2019",
    "Score": "avoi_score_2019",
    "Rank": "avoi_rank_2019"
})

africa_visa_openness_2023 = africa_visa_openness_2023.rename(columns={
    "Country": "country",
    "ISO3": "iso3",
    "No visa required": "avoi_no_visa_2023",
    "Visa on arrival": "avoi_visa_on_arrival_2023",
    "Visa required": "avoi_visa_required_2023",
    "Score": "avoi_score_2023",
    "Rank": "avoi_rank_2023"
})

passport_index_2019 = passport_index_2019.rename(columns={
    "Passport": "origin_iso3",
    "Destination": "destination_iso3",
    "Code": "code",
})

passport_index_2023 = passport_index_2023.rename(columns={
    "Passport": "origin_iso3",
    "Destination": "destination_iso3",
    "Requirement": "requirement",
})

henley_passport_index_2025 = henley_passport_index_2025.rename(columns={
    "Passport": "country",
    "ISO3": "iso3",
    "Visa-free score": "henley_score_2025",
    "Rank": "henley_rank_2025"
})

gdp_per_capita_2019_2023 = gdp_per_capita_2019_2023.rename(columns={
    "Country Code": "iso3",
    "2019": "gdp_ppc_2019",
    "2020": "gdp_ppc_2020",
    "2021": "gdp_ppc_2021",
    "2022": "gdp_ppc_2022",
    "2023": "gdp_ppc_2023"
})

country_region_income_group = country_region_income_group.rename(columns={
    "Country Code": "iso3",
    "Region": "region",
    "IncomeGroup": "income_group"
})

In [4]:
# Recode passport index
code_to_requirement_2019 = {
    3: 'visa free',
    2: 'visa on arrival',
    1: 'e-visa',
    0: 'visa required',
   -1: 'self-passport'
}

requirement_to_code_2023 = {
    'visa free': 3,
    'visa on arrival': 2,
    'e-visa': 1,
    'visa required': 0,
    'covid ban': -2,
    'no admission': -3,
    'Hayya Entry Permit': -4,
    '-1': -1
}

passport_index_2019["requirement"] = passport_index_2019["code"].map(code_to_requirement_2019).astype("category")
passport_index_2023["code"] = passport_index_2023["requirement"].map(requirement_to_code_2023).astype("Int64")

In [5]:
# Build graphs for passport index
graph_2019 = nx.DiGraph()
graph_2023 = nx.DiGraph()

# Only include edges where the code is greater than zero
for _, row in passport_index_2019[passport_index_2019["code"] > 0].iterrows():
    origin = row["origin_iso3"]
    destination = row["destination_iso3"]
    graph_2019.add_edge(origin, destination)

for _, row in passport_index_2023[passport_index_2023["code"] > 0].iterrows():
    origin = row["origin_iso3"]
    destination = row["destination_iso3"]
    graph_2023.add_edge(origin, destination)

# Compute centrality measures
katz_2019 = nx.katz_centrality_numpy(graph_2019, alpha=0.005, beta=1.0)
pagerank_2019 = nx.pagerank(graph_2019)
betweenness_2019 = nx.betweenness_centrality(graph_2019)
closeness_2019 = nx.closeness_centrality(graph_2019)

katz_2023 = nx.katz_centrality_numpy(graph_2023, alpha=0.005, beta=1.0)
pagerank_2023 = nx.pagerank(graph_2023)
betweenness_2023 = nx.betweenness_centrality(graph_2023)
closeness_2023 = nx.closeness_centrality(graph_2023)

nodes_2019 = list(graph_2019.nodes())
nodes_2023 = list(graph_2023.nodes())

centrality_2019 = pd.DataFrame({
    "iso3": nodes_2019,
    "passport_index_katz_2019": [katz_2019.get(n, None) for n in nodes_2019],
    "passport_index_pagerank_2019": [pagerank_2019.get(n, None) for n in nodes_2019],
    "passport_index_betweenness_2019": [betweenness_2019.get(n, None) for n in nodes_2019],
    "passport_index_closeness_2019": [closeness_2019.get(n, None) for n in nodes_2019],
})

centrality_2023 = pd.DataFrame({
    "iso3": nodes_2023,
    "passport_index_katz_2023": [katz_2023.get(n, None) for n in nodes_2023],
    "passport_index_pagerank_2023": [pagerank_2023.get(n, None) for n in nodes_2023],
    "passport_index_betweenness_2023": [betweenness_2023.get(n, None) for n in nodes_2023],
    "passport_index_closeness_2023": [closeness_2023.get(n, None) for n in nodes_2023]
})

In [6]:
# Merge datasets
merged = (africa_visa_openness_2019
    .merge(africa_visa_openness_2023, on="iso3", how="outer", suffixes=("", "_dup1"))
    .merge(henley_passport_index_2025, on="iso3", how="outer", suffixes=("", "_dup2"))
    .merge(centrality_2019, on="iso3", how="outer")
    .merge(centrality_2023, on="iso3", how="outer")
    .merge(gdp_per_capita_2019_2023, on="iso3", how="outer")
    .merge(country_region_income_group, on="iso3", how="left")
)

In [7]:
# Handle duplicate country names
merged["country"] = (
    merged["country"]
    .combine_first(merged.get("country_dup1"))
    .combine_first(merged.get("country_dup2"))
)

In [8]:
# Drop duplicate columns
merged = merged.drop(columns=[col for col in merged.columns if col.startswith("country_dup")])

In [9]:
# Drop duplicate rows
merged = merged.drop_duplicates()

In [10]:
# Remove rows with no country, iso3, or region
merged_cleaned = merged.dropna(subset=["country", "iso3", "region"])

In [11]:
# Remove rows with no GDP per capita
merged_cleaned = merged_cleaned.dropna(subset=["gdp_ppc_2019", "gdp_ppc_2020", "gdp_ppc_2021", "gdp_ppc_2022", "gdp_ppc_2023"])

In [12]:
# Remove GDP columns other than 2019 and 2023
merged_cleaned = merged_cleaned.drop(columns=["gdp_ppc_2020", "gdp_ppc_2021", "gdp_ppc_2022"])

In [15]:
# Add log GDP per capita
merged_cleaned["log_gdp_ppc_2019"] = np.log(merged_cleaned["gdp_ppc_2019"])
merged_cleaned["log_gdp_ppc_2023"] = np.log(merged_cleaned["gdp_ppc_2023"])

In [16]:
# Adding differences between 2023 and 2019
merged_cleaned["avoi_no_visa_change"] = merged_cleaned["avoi_no_visa_2023"] - merged_cleaned["avoi_no_visa_2019"]
merged_cleaned["avoi_visa_on_arrival_change"] = merged_cleaned["avoi_visa_on_arrival_2023"] - merged_cleaned["avoi_visa_on_arrival_2019"]
merged_cleaned["avoi_visa_required_change"] = merged_cleaned["avoi_visa_required_2023"] - merged_cleaned["avoi_visa_required_2019"]
merged_cleaned["avoi_score_change"] = merged_cleaned["avoi_score_2023"] - merged_cleaned["avoi_score_2019"]
merged_cleaned["avoi_rank_change"] = merged_cleaned["avoi_rank_2023"] - merged_cleaned["avoi_rank_2019"]
merged_cleaned["gdp_ppc_change"] = merged_cleaned["gdp_ppc_2023"] - merged_cleaned["gdp_ppc_2019"]
merged_cleaned["log_gdp_ppc_change"] = merged_cleaned["log_gdp_ppc_2023"] - merged_cleaned["log_gdp_ppc_2019"]

merged_cleaned["passport_index_katz_change"] = merged_cleaned["passport_index_katz_2023"] - merged_cleaned["passport_index_katz_2019"]
merged_cleaned["passport_index_pagerank_change"] = merged_cleaned["passport_index_pagerank_2023"] - merged_cleaned["passport_index_pagerank_2019"]
merged_cleaned["passport_index_betweenness_change"] = merged_cleaned["passport_index_betweenness_2023"] - merged_cleaned["passport_index_betweenness_2019"]
merged_cleaned["passport_index_closeness_change"] = merged_cleaned["passport_index_closeness_2023"] - merged_cleaned["passport_index_closeness_2019"]


In [17]:
merged_cleaned.head()

,country,avoi_no_visa_2019,avoi_visa_on_arrival_2019,avoi_visa_required_2019,avoi_score_2019,avoi_rank_2019,iso3,avoi_no_visa_2023,avoi_visa_on_arrival_2023,avoi_visa_required_2023,...,avoi_visa_on_arrival_change,avoi_visa_required_change,avoi_score_change,avoi_rank_change,gdp_ppc_change,log_gdp_ppc_change,passport_index_katz_change,passport_index_pagerank_change,passport_index_betweenness_change,passport_index_closeness_change
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,AFG,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-934.820750,-0.384710,0.010853,0.000000,0.000000,0.000000
4,Angola,10.0,7.0,36.0,0.294,34.0,AGO,10.0,0.0,43.0,...,-7.0,7.0,-0.105,7.0,-1029.649509,-0.132887,-0.011307,-0.002276,-0.000179,-0.116047
5,Albania,NaN,NaN,NaN,NaN,NaN,ALB,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2910.048820,0.176602,0.023626,0.007326,0.001336,0.044049
6,Andorra,NaN,NaN,NaN,NaN,NaN,AND,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1415.396600,0.022143,-0.012424,-0.002107,-0.000261,-0.274880
8,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ARE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-310.304840,-0.004515,-0.011175,-0.001240,0.004617,-0.112997


In [19]:
merged_cleaned.describe()

,avoi_no_visa_2019,avoi_visa_on_arrival_2019,avoi_visa_required_2019,avoi_score_2019,avoi_rank_2019,avoi_no_visa_2023,avoi_visa_on_arrival_2023,avoi_visa_required_2023,avoi_score_2023,avoi_rank_2023,...,avoi_visa_on_arrival_change,avoi_visa_required_change,avoi_score_change,avoi_rank_change,gdp_ppc_change,log_gdp_ppc_change,passport_index_katz_change,passport_index_pagerank_change,passport_index_betweenness_change,passport_index_closeness_change
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,...,52.000000,52.000000,52.000000,52.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000
mean,13.500000,14.403846,24.961538,0.474712,26.326923,15.192308,14.211538,23.596154,0.501173,26.307692,...,-0.192308,-1.365385,0.026462,-0.019231,893.365395,0.030854,-0.001865,-0.000027,0.001112,-0.167755
std,10.861878,18.870310,19.184309,0.304847,15.415451,13.244368,18.693961,18.955324,0.308059,15.549464,...,12.541755,11.575886,0.178786,8.779411,4092.963250,0.133710,0.023794,0.004189,0.010211,0.279225
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.019000,1.000000,...,-36.000000,-47.000000,-0.476000,-34.000000,-28587.544500,-0.384710,-0.056361,-0.010051,-0.015792,-1.000000
25%,6.000000,0.000000,1.000000,0.256250,13.750000,6.750000,0.000000,1.000000,0.251750,13.750000,...,-0.250000,-1.000000,-0.001000,-0.250000,-91.698706,-0.009871,-0.012929,-0.002131,-0.000941,-0.277754
50%,14.000000,2.000000,31.500000,0.402000,26.500000,14.000000,2.000000,29.500000,0.422500,26.500000,...,0.000000,0.000000,0.000000,1.000000,349.553402,0.032311,-0.004099,-0.000699,-0.000133,-0.120508
75%,17.000000,32.500000,39.250000,0.805750,39.250000,19.000000,26.000000,39.250000,0.802750,39.250000,...,0.000000,0.250000,0.019000,3.000000,1528.000647,0.078930,0.008961,0.001826,0.001873,-0.005025
max,53.000000,53.000000,53.000000,1.000000,54.000000,53.000000,53.000000,52.000000,1.000000,54.000000,...,47.000000,32.000000,0.710000,25.000000,35912.738800,1.302796,0.074349,0.011883,0.086619,0.528143


In [31]:
# Create a subset for each of these regions 'Sub-Saharan Africa' 'Middle East & North Africa'
merged_cleaned_sub_saharan = merged_cleaned[merged_cleaned["region"] == "Sub-Saharan Africa"]
merged_cleaned_north_africa = merged_cleaned[merged_cleaned["region"] == "Middle East & North Africa"]

In [22]:
list(merged_cleaned.columns)

['country',
 'avoi_no_visa_2019',
 'avoi_visa_on_arrival_2019',
 'avoi_visa_required_2019',
 'avoi_score_2019',
 'avoi_rank_2019',
 'iso3',
 'avoi_no_visa_2023',
 'avoi_visa_on_arrival_2023',
 'avoi_visa_required_2023',
 'avoi_score_2023',
 'avoi_rank_2023',
 'henley_rank_2025',
 'henley_score_2025',
 'passport_index_katz_2019',
 'passport_index_pagerank_2019',
 'passport_index_betweenness_2019',
 'passport_index_closeness_2019',
 'passport_index_katz_2023',
 'passport_index_pagerank_2023',
 'passport_index_betweenness_2023',
 'passport_index_closeness_2023',
 'gdp_ppc_2019',
 'gdp_ppc_2023',
 'region',
 'income_group',
 'log_gdp_ppc_2019',
 'log_gdp_ppc_2023',
 'avoi_no_visa_change',
 'avoi_visa_on_arrival_change',
 'avoi_visa_required_change',
 'avoi_score_change',
 'avoi_rank_change',
 'gdp_ppc_change',
 'log_gdp_ppc_change',
 'passport_index_katz_change',
 'passport_index_pagerank_change',
 'passport_index_betweenness_change',
 'passport_index_closeness_change']

In [21]:
# regress log_gdp_ppc_2023 on avoi_score_2023 
smf.ols(formula="log_gdp_ppc_2023 ~ avoi_score_2023", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.135
Date:                Wed, 26 Mar 2025   Prob (F-statistic):              0.292
Time:                        05:52:07   Log-Likelihood:                -65.658
No. Observations:                  52   AIC:                             135.3
Df Residuals:                      50   BIC:                             139.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.6449      0.233     37.162      0.000       8.178       9.112
avoi_score_2023    -0.4223      0.396     -1.065      0.292      -1.219       0.374
==============================================================================
Omnibus:                        1.134   Durbin-Watson:                   2.447
Prob(Omnibus):                  0.567   Jarque-Bera (JB):                1.156
Skew:                           0.259   Prob(JB):                        0.561
Kurtosis:                       2.485   Cond. No.                         4.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# regress log_gdp_ppc_2023 on avoi_score_2023 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ avoi_score_2023 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     46.95
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           7.57e-16
Time:                        05:54:24   Log-Likelihood:                -24.417
No. Observations:                  52   AIC:                             58.83
Df Residuals:                      47   BIC:                             68.59
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.3344      0.452     22.840      0.000       9.424      11.245
income_group[T.Low income]             -2.7231      0.426     -6.388      0.000      -3.581      -1.866
income_group[T.Lower middle income]    -1.6535      0.427     -3.869      0.000      -2.513      -0.794
income_group[T.Upper middle income]    -0.6543      0.453     -1.445      0.155      -1.565       0.256
avoi_score_2023                        -0.0433      0.198     -0.219      0.828      -0.441       0.354
==============================================================================
Omnibus:                        0.294   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.863   Jarque-Bera (JB):                0.195
Skew:                           0.144   Prob(JB):                        0.907
Kurtosis:                       2.917   Cond. No.                         19.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
# regress log_gdp_ppc_2023 on avoi_score_2023 and income_group for only sub-Saharan Africa
smf.ols(formula="log_gdp_ppc_2023 ~ avoi_score_2023 + income_group", data=merged_cleaned_sub_saharan).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.817
Method:                 Least Squares   F-statistic:                     51.14
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.09e-15
Time:                        06:01:33   Log-Likelihood:                -16.195
No. Observations:                  46   AIC:                             42.39
Df Residuals:                      41   BIC:                             51.53
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.1766      0.411     24.761      0.000       9.347      11.007
income_group[T.Low income]             -2.6528      0.383     -6.927      0.000      -3.426      -1.879
income_group[T.Lower middle income]    -1.7032      0.384     -4.430      0.000      -2.480      -0.927
income_group[T.Upper middle income]    -0.5090      0.410     -1.241      0.222      -1.338       0.320
avoi_score_2023                         0.1145      0.190      0.603      0.550      -0.269       0.498
==============================================================================
Omnibus:                        1.005   Durbin-Watson:                   2.145
Prob(Omnibus):                  0.605   Jarque-Bera (JB):                1.058
Skew:                          -0.301   Prob(JB):                        0.589
Kurtosis:                       2.564   Cond. No.                         18.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# regress log_gdp_ppc_2023 on avoi_score_2023 and income_group for only North Africa
# This may be because of Egypt's high GDP per capita and low visa openness
smf.ols(formula="log_gdp_ppc_2023 ~ avoi_score_2023 + income_group ", data=merged_cleaned_north_africa).fit().summary()

/Users/mtwesley/.pyenv/versions/anaconda3-2023.09-0/envs/cs7280/lib/python3.13/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.581
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     2.079
Date:                Wed, 26 Mar 2025   Prob (F-statistic):              0.271
Time:                        06:02:25   Log-Likelihood:                0.80869
No. Observations:                   6   AIC:                             4.383
Df Residuals:                       3   BIC:                             3.758
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.3939      0.115     55.453      0.000       6.027       6.761
income_group[T.Lower middle income]     3.1967      0.169     18.888      0.000       2.658       3.735
income_group[T.Upper middle income]     3.1972      0.155     20.629      0.000       2.704       3.690
avoi_score_2023                        -0.9399      0.540     -1.740      0.180      -2.659       0.779
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.067
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.555
Skew:                          -0.456   Prob(JB):                        0.758
Kurtosis:                       1.822   Cond. No.                     1.93e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.66e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# regress log_gdp_ppc_2023 on henley_score_2025
smf.ols(formula="log_gdp_ppc_2023 ~ henley_score_2025", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     253.5
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.54e-36
Time:                        06:02:41   Log-Likelihood:                -209.01
No. Observations:                 184   AIC:                             422.0
Df Residuals:                     182   BIC:                             428.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             7.7489      0.128     60.461      0.000       7.496       8.002
henley_score_2025     0.0166      0.001     15.922      0.000       0.015       0.019
==============================================================================
Omnibus:                        0.417   Durbin-Watson:                   2.174
Prob(Omnibus):                  0.812   Jarque-Bera (JB):                0.236
Skew:                          -0.079   Prob(JB):                        0.889
Kurtosis:                       3.074   Cond. No.                         282.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
# regress log_gdp_ppc_2023 on henley_score_2025 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ henley_score_2025 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     314.0
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           9.71e-80
Time:                        06:03:44   Log-Likelihood:                -97.769
No. Observations:                 184   AIC:                             205.5
Df Residuals:                     179   BIC:                             221.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.5477      0.176     60.100      0.000      10.201      10.894
income_group[T.Low income]             -3.0459      0.154    -19.746      0.000      -3.350      -2.742
income_group[T.Lower middle income]    -1.9670      0.128    -15.406      0.000      -2.219      -1.715
income_group[T.Upper middle income]    -0.9379      0.099     -9.480      0.000      -1.133      -0.743
henley_score_2025                       0.0016      0.001      1.572      0.118      -0.000       0.004
==============================================================================
Omnibus:                        3.383   Durbin-Watson:                   2.327
Prob(Omnibus):                  0.184   Jarque-Bera (JB):                3.057
Skew:                          -0.220   Prob(JB):                        0.217
Kurtosis:                       3.453   Cond. No.                     1.04e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
# regress log_gdp_ppc_2023 on henley_score_2025 and region
smf.ols(formula="log_gdp_ppc_2023 ~ henley_score_2025 + region", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.708
Method:                 Least Squares   F-statistic:                     64.37
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           3.02e-45
Time:                        06:04:15   Log-Likelihood:                -172.46
No. Observations:                 184   AIC:                             360.9
Df Residuals:                     176   BIC:                             386.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                8.0011      0.181     44.090      0.000       7.643       8.359
region[T.Europe & Central Asia]          0.4435      0.152      2.923      0.004       0.144       0.743
region[T.Latin America & Caribbean]      0.0009      0.163      0.006      0.995      -0.320       0.322
region[T.Middle East & North Africa]     0.9098      0.193      4.723      0.000       0.530       1.290
region[T.North America]                  0.5979      0.468      1.277      0.203      -0.326       1.522
region[T.South Asia]                     0.2747      0.278      0.987      0.325      -0.275       0.824
region[T.Sub-Saharan Africa]            -0.5627      0.161     -3.497      0.001      -0.880      -0.245
henley_score_2025                        0.0136      0.001     11.297      0.000       0.011       0.016
==============================================================================
Omnibus:                        1.170   Durbin-Watson:                   2.420
Prob(Omnibus):                  0.557   Jarque-Bera (JB):                0.821
Skew:                          -0.027   Prob(JB):                        0.663
Kurtosis:                       3.323   Cond. No.                     1.25e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.25e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
# regress log_gdp_ppc_2023 on passport_index_katz_2023
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_katz_2023", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     40.77
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           1.38e-09
Time:                        06:05:15   Log-Likelihood:                -270.68
No. Observations:                 184   AIC:                             545.4
Df Residuals:                     182   BIC:                             551.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   10.9842      0.233     47.236      0.000      10.525      11.443
passport_index_katz_2023   -20.7310      3.247     -6.385      0.000     -27.137     -14.325
==============================================================================
Omnibus:                        5.636   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.060   Jarque-Bera (JB):                5.482
Skew:                          -0.422   Prob(JB):                       0.0645
Kurtosis:                       3.063   Cond. No.                         41.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
# regress log_gdp_ppc_2023 on passport_index_katz_2023 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_katz_2023 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     309.3
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           3.17e-79
Time:                        06:05:37   Log-Likelihood:                -98.987
No. Observations:                 184   AIC:                             208.0
Df Residuals:                     179   BIC:                             224.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.8351      0.099    109.522      0.000      10.640      11.030
income_group[T.Low income]             -3.2120      0.113    -28.509      0.000      -3.434      -2.990
income_group[T.Lower middle income]    -2.1127      0.089    -23.792      0.000      -2.288      -1.937
income_group[T.Upper middle income]    -1.0308      0.079    -13.011      0.000      -1.187      -0.874
passport_index_katz_2023               -0.4292      1.474     -0.291      0.771      -3.337       2.479
==============================================================================
Omnibus:                        3.891   Durbin-Watson:                   2.324
Prob(Omnibus):                  0.143   Jarque-Bera (JB):                3.600
Skew:                          -0.244   Prob(JB):                        0.165
Kurtosis:                       3.481   Cond. No.                         51.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [52]:
# regress log_gdp_ppc_2023 on passport_index_katz_2023 and income_group for only sub-Saharan Africa
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_katz_2023 + income_group", data=merged_cleaned_sub_saharan).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     50.90
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.27e-15
Time:                        06:40:02   Log-Likelihood:                -16.286
No. Observations:                  46   AIC:                             42.57
Df Residuals:                      41   BIC:                             51.72
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.3340      0.378     27.371      0.000       9.572      11.096
income_group[T.Low income]             -2.6623      0.386     -6.905      0.000      -3.441      -1.884
income_group[T.Lower middle income]    -1.7117      0.388     -4.410      0.000      -2.496      -0.928
income_group[T.Upper middle income]    -0.5445      0.402     -1.355      0.183      -1.356       0.267
passport_index_katz_2023               -1.0026      2.240     -0.448      0.657      -5.526       3.521
==============================================================================
Omnibus:                        1.213   Durbin-Watson:                   2.072
Prob(Omnibus):                  0.545   Jarque-Bera (JB):                1.157
Skew:                          -0.248   Prob(JB):                        0.561
Kurtosis:                       2.401   Cond. No.                         49.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
# regress log_gdp_ppc_2023 on passport_index_katz_2023 and region
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_katz_2023 + region", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.526
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     27.91
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           1.37e-25
Time:                        06:05:51   Log-Likelihood:                -220.58
No. Observations:                 184   AIC:                             457.2
Df Residuals:                     176   BIC:                             482.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                9.9889      0.253     39.516      0.000       9.490      10.488
region[T.Europe & Central Asia]          0.8332      0.194      4.298      0.000       0.451       1.216
region[T.Latin America & Caribbean]      0.1353      0.214      0.633      0.527      -0.286       0.557
region[T.Middle East & North Africa]     0.4602      0.245      1.881      0.062      -0.023       0.943
region[T.North America]                  1.4235      0.600      2.371      0.019       0.239       2.608
region[T.South Asia]                    -0.5384      0.352     -1.530      0.128      -1.233       0.156
region[T.Sub-Saharan Africa]            -1.1798      0.198     -5.966      0.000      -1.570      -0.790
passport_index_katz_2023                -5.8939      2.963     -1.989      0.048     -11.741      -0.046
==============================================================================
Omnibus:                        0.098   Durbin-Watson:                   2.141
Prob(Omnibus):                  0.952   Jarque-Bera (JB):                0.089
Skew:                           0.050   Prob(JB):                        0.956
Kurtosis:                       2.959   Cond. No.                         53.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
# regress log_gdp_ppc_2023 on passport_index_katz_2023 and income_group and region
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_katz_2023 + income_group + region", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     137.1
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           9.91e-77
Time:                        06:06:12   Log-Likelihood:                -87.931
No. Observations:                 184   AIC:                             197.9
Df Residuals:                     173   BIC:                             233.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               10.6312      0.129     82.187      0.000      10.376      10.887
income_group[T.Low income]              -2.9764      0.141    -21.164      0.000      -3.254      -2.699
income_group[T.Lower middle income]     -1.9814      0.097    -20.524      0.000      -2.172      -1.791
income_group[T.Upper middle income]     -0.9318      0.080    -11.635      0.000      -1.090      -0.774
region[T.Europe & Central Asia]          0.2712      0.099      2.743      0.007       0.076       0.466
region[T.Latin America & Caribbean]     -0.0334      0.107     -0.311      0.756      -0.245       0.178
region[T.Middle East & North Africa]     0.2799      0.120      2.326      0.021       0.042       0.517
region[T.North America]                  0.4563      0.298      1.529      0.128      -0.133       1.045
region[T.South Asia]                     0.2528      0.176      1.437      0.153      -0.094       0.600
region[T.Sub-Saharan Africa]            -0.0752      0.113     -0.668      0.505      -0.297       0.147
passport_index_katz_2023                -0.0913      1.496     -0.061      0.951      -3.043       2.861
==============================================================================
Omnibus:                        0.894   Durbin-Watson:                   2.362
Prob(Omnibus):                  0.640   Jarque-Bera (JB):                0.551
Skew:                          -0.022   Prob(JB):                        0.759
Kurtosis:                       3.265   Cond. No.                         59.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
# regress log_gdp_ppc_2023 on passport_index_pagerank_2023
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_pagerank_2023", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     46.79
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           1.16e-10
Time:                        06:06:59   Log-Likelihood:                -268.23
No. Observations:                 184   AIC:                             540.5
Df Residuals:                     182   BIC:                             546.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       10.1628      0.114     88.931      0.000       9.937      10.388
passport_index_pagerank_2023  -111.3228     16.274     -6.840      0.000    -143.433     -79.213
==============================================================================
Omnibus:                        6.252   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                6.062
Skew:                          -0.441   Prob(JB):                       0.0483
Kurtosis:                       3.120   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
# regress log_gdp_ppc_2023 on passport_index_pagerank_2023 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_pagerank_2023 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     309.7
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.89e-79
Time:                        06:07:11   Log-Likelihood:                -98.894
No. Observations:                 184   AIC:                             207.8
Df Residuals:                     179   BIC:                             223.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.8213      0.057    189.650      0.000      10.709      10.934
income_group[T.Low income]             -3.2022      0.113    -28.227      0.000      -3.426      -2.978
income_group[T.Lower middle income]    -2.1026      0.090    -23.359      0.000      -2.280      -1.925
income_group[T.Upper middle income]    -1.0265      0.080    -12.858      0.000      -1.184      -0.869
passport_index_pagerank_2023           -3.8947      7.534     -0.517      0.606     -18.761      10.972
==============================================================================
Omnibus:                        3.716   Durbin-Watson:                   2.327
Prob(Omnibus):                  0.156   Jarque-Bera (JB):                3.399
Skew:                          -0.239   Prob(JB):                        0.183
Kurtosis:                       3.463   Cond. No.                         264.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
# regress log_gdp_ppc_2023 on passport_index_pagerank_2023 and income_group for only sub-Saharan Africa
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_pagerank_2023 + income_group", data=merged_cleaned_sub_saharan).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     50.79
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           2.35e-15
Time:                        06:40:40   Log-Likelihood:                -16.328
No. Observations:                  46   AIC:                             42.66
Df Residuals:                      41   BIC:                             51.80
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.2944      0.366     28.155      0.000       9.556      11.033
income_group[T.Low income]             -2.6706      0.386     -6.917      0.000      -3.450      -1.891
income_group[T.Lower middle income]    -1.7205      0.389     -4.424      0.000      -2.506      -0.935
income_group[T.Upper middle income]    -0.5484      0.404     -1.358      0.182      -1.364       0.267
passport_index_pagerank_2023           -4.3407     12.243     -0.355      0.725     -29.066      20.385
==============================================================================
Omnibus:                        1.130   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.568   Jarque-Bera (JB):                1.108
Skew:                          -0.245   Prob(JB):                        0.575
Kurtosis:                       2.418   Cond. No.                         268.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
# regress log_gdp_ppc_2023 on passport_index_pagerank_2023 and region
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_pagerank_2023 + region", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     28.21
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           8.47e-26
Time:                        06:07:20   Log-Likelihood:                -220.07
No. Observations:                 184   AIC:                             456.1
Df Residuals:                     176   BIC:                             481.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                9.7705      0.171     57.050      0.000       9.433      10.109
region[T.Europe & Central Asia]          0.8103      0.195      4.161      0.000       0.426       1.195
region[T.Latin America & Caribbean]      0.1402      0.212      0.661      0.509      -0.278       0.559
region[T.Middle East & North Africa]     0.4426      0.244      1.814      0.071      -0.039       0.924
region[T.North America]                  1.3748      0.600      2.291      0.023       0.191       2.559
region[T.South Asia]                    -0.5396      0.349     -1.545      0.124      -1.229       0.150
region[T.Sub-Saharan Africa]            -1.1736      0.197     -5.964      0.000      -1.562      -0.785
passport_index_pagerank_2023           -33.3808     14.982     -2.228      0.027     -62.948      -3.814
==============================================================================
Omnibus:                        0.214   Durbin-Watson:                   2.149
Prob(Omnibus):                  0.899   Jarque-Bera (JB):                0.136
Skew:                           0.066   Prob(JB):                        0.934
Kurtosis:                       3.004   Cond. No.                         270.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
# regress log_gdp_ppc_2023 on passport_index_pagerank_2023 and income_group and region
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_pagerank_2023 + income_group + region", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     137.1
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           9.92e-77
Time:                        06:07:27   Log-Likelihood:                -87.932
No. Observations:                 184   AIC:                             197.9
Df Residuals:                     173   BIC:                             233.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               10.6268      0.095    112.109      0.000      10.440      10.814
income_group[T.Low income]              -2.9765      0.141    -21.142      0.000      -3.254      -2.699
income_group[T.Lower middle income]     -1.9818      0.097    -20.414      0.000      -2.173      -1.790
income_group[T.Upper middle income]     -0.9317      0.080    -11.587      0.000      -1.090      -0.773
region[T.Europe & Central Asia]          0.2714      0.099      2.734      0.007       0.075       0.467
region[T.Latin America & Caribbean]     -0.0330      0.107     -0.308      0.758      -0.244       0.178
region[T.Middle East & North Africa]     0.2797      0.120      2.325      0.021       0.042       0.517
region[T.North America]                  0.4563      0.299      1.527      0.129      -0.133       1.046
region[T.South Asia]                     0.2521      0.175      1.438      0.152      -0.094       0.598
region[T.Sub-Saharan Africa]            -0.0756      0.112     -0.673      0.502      -0.298       0.146
passport_index_pagerank_2023            -0.3055      7.612     -0.040      0.968     -15.329      14.718
==============================================================================
Omnibus:                        0.896   Durbin-Watson:                   2.362
Prob(Omnibus):                  0.639   Jarque-Bera (JB):                0.553
Skew:                          -0.022   Prob(JB):                        0.759
Kurtosis:                       3.265   Cond. No.                         299.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
# regress log_gdp_ppc_2023 on passport_index_betweenness_2023 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_betweenness_2023 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     312.2
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           1.54e-79
Time:                        06:38:47   Log-Likelihood:                -98.241
No. Observations:                 184   AIC:                             206.5
Df Residuals:                     179   BIC:                             222.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.8295      0.055    196.087      0.000      10.720      10.938
income_group[T.Low income]             -3.2338      0.106    -30.508      0.000      -3.443      -3.025
income_group[T.Lower middle income]    -2.1322      0.080    -26.562      0.000      -2.291      -1.974
income_group[T.Upper middle income]    -1.0386      0.079    -13.175      0.000      -1.194      -0.883
passport_index_betweenness_2023        -3.7896      3.050     -1.242      0.216      -9.809       2.230
==============================================================================
Omnibus:                        2.916   Durbin-Watson:                   2.340
Prob(Omnibus):                  0.233   Jarque-Bera (JB):                2.557
Skew:                          -0.199   Prob(JB):                        0.279
Kurtosis:                       3.418   Cond. No.                         107.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
# regress log_gdp_ppc_2023 on passport_index_closeness_2023 and income_group
smf.ols(formula="log_gdp_ppc_2023 ~ passport_index_closeness_2023 + income_group", data=merged_cleaned).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       log_gdp_ppc_2023   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     309.2
Date:                Wed, 26 Mar 2025   Prob (F-statistic):           3.23e-79
Time:                        06:39:06   Log-Likelihood:                -99.007
No. Observations:                 184   AIC:                             208.0
Df Residuals:                     179   BIC:                             224.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              10.7994      0.075    144.218      0.000      10.652      10.947
income_group[T.Low income]             -3.2303      0.111    -29.045      0.000      -3.450      -3.011
income_group[T.Lower middle income]    -2.1295      0.085    -25.145      0.000      -2.297      -1.962
income_group[T.Upper middle income]    -1.0326      0.079    -13.069      0.000      -1.189      -0.877
passport_index_closeness_2023           0.0250      0.115      0.218      0.828      -0.202       0.252
==============================================================================
Omnibus:                        4.164   Durbin-Watson:                   2.317
Prob(Omnibus):                  0.125   Jarque-Bera (JB):                3.923
Skew:                          -0.252   Prob(JB):                        0.141
Kurtosis:                       3.508   Cond. No.                         5.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""